<span style="font-size:200%">Técnicas de *grid*</span><br>
<span style="color: gray">dic 2019</span><br>
[*Alberto Ruiz*](http://dis.um.es/profesores/alberto)

La inferencia Bayesiana puede resolverse mediante exploración exhaustiva cuando el número de parámetros es pequeño.

## Normal con outliers

### Normal con outliers

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib import rc
#rc('text', usetex=True)
#rc('font', size=14)

G = np.random.randn

def show1ddata(x, sz=(8,0.5), **kwargs):
    plt.figure(figsize=sz)
    
    options = { 'marker': 'x', 's': 60, 'alpha': 0.75, 'color':'blue' }
    options.update(kwargs)
    
    plt.ylim(-1,1);
    plt.scatter(x,x*0, zorder=5, **options);
    
    ax = plt.gca()
    ax.spines['left'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_yticklabels([])
    ax.set_yticks([])
    ax.spines['bottom'].set_position('zero')
    ax.spines['bottom'].set_color('gray')
    
    col = 'gray'
    ax.tick_params(axis='x', colors=col)
    
def shbracket(x, k=2):
    m = x.mean()
    s = x.std()
    d = 0.6
    plt.fill_between([m-k*s,m+k*s], [d,d], -d, alpha=0.2, color='green')
    
def shrange(x1,x2,d):
    plt.fill_between([x1,x2], [d,d], -d, alpha=0.2, color='green')

Partimos de una muestra de una variable aleatoria gaussiana.

In [ ]:
n = 10

μ = 1
σ = 0.5

data = μ  +  σ * G(n)
#print(data)
show1ddata(data, color='blue', alpha=0.5)
plt.xlim(-5,5);

Lo usual es calcular la media y la desviación estándar. Cuanto mayor sea $n$ más se aproximarán al valor real.

In [ ]:
print(np.mean(data), np.std(data))
show1ddata(data, color='blue', alpha=0.5)
plt.xlim(-5,5);
shbracket(data)

A veces los datos disponibles están contaminados con  "outliers", y lo que es peor, es posible que la proporción  $\varepsilon$ de dichos outliers sea desconocida.

In [ ]:
noisydata = np.append(data,[-4.3, 2.2, 4.1])

show1ddata(noisydata, color='blue', alpha=0.5)
plt.xlim(-5,5);
shbracket(noisydata)

In [ ]:
from numpy import log

def normalize(x):
    return x/x.sum()

def lgauss1d(m, s, x):
    return -0.5 * ((x-m)/s)**2 - log(s)


def ljeffreys(s):
    return -log(s) if s > 0 else MINF


def logprob(D):
    def f(θ):
        m,s = θ
        return sum(lgauss1d(m,s,D)) + ljeffreys(s)
    return f

Discretizamos los parámetros.

In [ ]:
n = 100
M = np.linspace(0,2,n)
S = np.linspace(0.1,1,n+1)

Evaluamos la verosimilitud $\times$ prior de todas las configuraciones, y al normalizar tenemos las probabilidades a posteriori.

In [ ]:
f = logprob(data)

P = normalize(np.exp([[f((m,s)) for m in M] for s in S]))

Mostramos gráficamente la densidad conjunta:

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

m,s = np.meshgrid(M,S)

fig = plt.figure(figsize=(12,4))
ax = fig.add_subplot(121, projection='3d')
ax.plot_surface(m,s,P, cmap='coolwarm', linewidth=0.5, rstride=2, cstride=2);
ax.set_zticks([]);
ax.set_xticks(np.linspace(0,2,5))
ax.set_title('$P(\mu \sigma| D)$');
ax.set_xlabel('$\mu$'); ax.set_ylabel('$\sigma$');

plt.subplot(1,2,2)
plt.imshow(-np.flipud(P),'gray');
ax = plt.gca()
ax.set_xticks([0,len(M)-1]); ax.set_xticklabels(M[[0,-1]]);
plt.xlabel('$\mu$')
ax.set_yticks([0,len(S)-1]); ax.set_yticklabels(S[[-1,0]]);
plt.ylabel('$\sigma$');

In [ ]:
def levels(P, probs=[0.99, 0.9, 0.5]):
    vals = np.sort(P.flatten())[::-1]
    cum  = np.cumsum(vals)
    v = vals[[np.where(cum > p)[0][0] for p in probs ]]
    fmt = {v:f'{100*p:.0f}%' for v,p in zip(v,probs)}
    return v,fmt

Es más informativo mostrar las curvas de nivel que engloban diferentes cantidades de probabilidad acumulada:

In [ ]:
plt.figure(figsize=(5,5))
lev,fmt = levels(P, probs=[0.99,0.95, 0.9,0.8, 0.5])

CS = plt.contour(M,S,P,colors='black',levels=lev);
plt.grid(ls='dashed');plt.xlabel('$\mu$'); plt.ylabel('$\sigma$');
plt.clabel(CS,CS.levels,fmt=fmt,fontsize=10);
plt.title('$\mathcal{P}(\,\mu,\sigma\;|\;D\,)$',fontsize=16);

Las dos densidades marginales son:

In [ ]:
plt.figure(figsize=(12,4))

plt.subplot(1,2,1);
plt.plot(M, P.sum(axis=0)); plt.xlabel('$\mu$'); plt.yticks([]); plt.title('$\mathcal{P}(\,\mu\;|\;D\,)$')

plt.subplot(1,2,2);
plt.plot(S, P.sum(axis=1)); plt.xlabel('$\sigma$'); plt.yticks([]); plt.title('$\mathcal{P}(\,\sigma\;|\;D\,)$');

Ampliamos el modelo con la proporción de outliers. Se trata de una mezcla de la gaussiana "real" con otra muy ancha que recoge los outliers.

In [ ]:
def gaussian1d(m,s,x):
    return 1/np.sqrt(2*np.pi)/s * np.exp ( -0.5 * ((x-m)/s)**2 )

def rmod(e,m,s,x):
    return log( (1-e)*gaussian1d(m ,s, x) + e* gaussian1d(0, 5, x) )

Calculamos el bloque de probabilidades a posteriori, ahora 3D. 

In [ ]:
n = 100
M = np.linspace(0,2,n)
S = np.linspace(0.1,1,n+1)
E = np.linspace(0.01,0.99,n+2)

In [ ]:
# Requiere bastante más tiempo de cómputo al hacerlo con listas de Python.
# Aprox 1m vs menos de 2s usando la vectorización de np
# ¿Cuánto tarda con GPU?

if False:
    
    def logprob(D):
        def f(θ):
            e,m,s = θ
            return sum(rmod(e,m,s,D)) + ljeffreys(s) # + 0 + lunif(0,1,p)
        return f

    f = logprob(data)

    P = normalize(np.exp([[[f((e,m,s)) for m in M] for s in S] for e in E]))

    plt.figure(figsize=(5,5))
    plt.contour(M,S,P.sum(axis=0),colors='black'); plt.grid(ls='dashed');plt.xlabel('$\mu$'); plt.ylabel('$\sigma$');
    plt.title('$\mathcal{P}\,(\,\mu,\sigma\;|\;D\,)$',fontsize=16);

In [ ]:
# aprovechamos la vectorización de numpy

e,s,m = np.meshgrid(E,S,M,indexing='ij')

# En vez de esta suma explícita se puede crear un bloque con todos los datos, 
# expandiendo adecuadamente las dimensiones, y efectuando la suma con np, pero
# me ha resultado menos eficiente

L = sum([rmod(e,m,s,d) for d in data]) - log(s)
P = normalize(np.exp(L))

Pms = P.sum(axis=0)

plt.figure(figsize=(5,5))

lev,fmt = levels(Pms, probs=[0.99, 0.9, 0.5])
CS = plt.contour(M,S,Pms,colors='black',levels=lev);
plt.clabel(CS,CS.levels,fmt=fmt,fontsize=10);
    
plt.grid(ls='dashed');plt.xlabel('$\mu$'); plt.ylabel('$\sigma$');
plt.title('$\mathcal{P}(\,\mu,\sigma\;|\;D\,)$',fontsize=16);

In [ ]:
Pes = P.sum(axis=2)

plt.figure(figsize=(5,5))

lev,fmt = levels(Pes, probs=[0.99, 0.9, 0.5])
CS = plt.contour(S,E,Pes,colors='black',levels=lev);
plt.clabel(CS,CS.levels,fmt=fmt,fontsize=10);
    
plt.grid(ls='dashed');plt.xlabel('$\sigma$'); plt.ylabel('$\epsilon$');
plt.title('$\mathcal{P}(\,\epsilon,\sigma\;|\;D\,)$',fontsize=16);

In [ ]:
plt.figure(figsize=(15,4))

plt.subplot(1,3,1);
plt.plot(M, P.sum(axis=(0,1))); plt.xlabel('$\mu$'); plt.yticks([]); plt.title('$\mathcal{P}(\,\mu\;|\;D\,)$')

plt.subplot(1,3,2);
plt.plot(S, P.sum(axis=(0,2))); plt.xlabel('$\sigma$'); plt.yticks([]); plt.title('$\mathcal{P}(\,\sigma\;|\;D\,)$');

plt.subplot(1,3,3)
plt.plot(E, P.sum(axis=(1,2))); plt.xlabel('$\epsilon$');  plt.yticks([]); plt.title('$\mathcal{P}(\,\epsilon\;|\;D\,)$');

Repetimos el proceso, pero ahora con los datos contaminados. Aprovechamos para medir el tiempo de cálculo.

In [ ]:
L = sum([rmod(e,m,s,d) for d in noisydata]) - log(s)
Pn = normalize(np.exp(L))

Pms = Pn.sum(axis=0)

In [ ]:
plt.figure(figsize=(5,5))

lev,fmt = levels(Pms, probs=[0.99, 0.9, 0.5])
CS = plt.contour(M,S,Pms,colors='black',levels=lev);
plt.clabel(CS,CS.levels,fmt=fmt,fontsize=10);
    
plt.grid(ls='dashed');plt.xlabel('$\mu$'); plt.ylabel('$\sigma$');
plt.title('$\mathcal{P}(\,\mu,\sigma\;|\;D\,)$',fontsize=16);

In [ ]:
Pes = Pn.sum(axis=2)

plt.figure(figsize=(5,5))

lev,fmt = levels(Pes, probs=[0.99, 0.9, 0.5])
CS = plt.contour(S,E,Pes,colors='black',levels=lev);
plt.clabel(CS,CS.levels,fmt=fmt,fontsize=10);
    
plt.grid(ls='dashed');plt.xlabel('$\sigma$'); plt.ylabel('$\epsilon$');
plt.title('$\mathcal{P}(\,\epsilon,\sigma\;|\;D\,)$',fontsize=16);

In [ ]:
plt.figure(figsize=(15,4))

plt.subplot(1,3,1);
plt.plot(M, Pn.sum(axis=(0,1))); plt.xlabel('$\mu$'); plt.yticks([]); plt.title('$\mathcal{P}(\,\mu\;|\;D\,)$')

plt.subplot(1,3,2);
plt.plot(S, Pn.sum(axis=(0,2))); plt.xlabel('$\sigma$'); plt.yticks([]); plt.title('$\mathcal{P}(\,\sigma\;|\;D\,)$');

plt.subplot(1,3,3)
plt.plot(E, Pn.sum(axis=(1,2))); plt.xlabel('$\epsilon$');  plt.yticks([]); plt.title('$\mathcal{P}(\,\epsilon\;|\;D\,)$');

In [ ]:
plt.figure(figsize=(15,4))

plt.subplot(1,3,1);
plt.plot(M, P.sum(axis=(0,1)));
plt.plot(M, Pn.sum(axis=(0,1))); plt.xlabel('$\mu$'); plt.yticks([]); plt.title('$\mathcal{P}(\,\mu\;|\;D\,)$')

plt.subplot(1,3,2);
plt.plot(S, P.sum(axis=(0,2)));
plt.plot(S, Pn.sum(axis=(0,2))); plt.xlabel('$\sigma$'); plt.yticks([]); plt.title('$\mathcal{P}(\,\sigma\;|\;D\,)$');

plt.subplot(1,3,3)
plt.plot(E, P.sum(axis=(1,2)));
plt.plot(E, Pn.sum(axis=(1,2))); plt.xlabel('$\epsilon$');  plt.yticks([]);  plt.title('$\mathcal{P}(\,\epsilon\;|\;D\,)$');

Los estimadores de $\mu$ y $\sigma$ son parecidos con y sin outliers, pero la existencia de outliers y su proporción son correctamente detectadas.

### Intervalo de confianza

Primero comprobamos el método tradicional para construir un [intervalo de confianza](https://en.wikipedia.org/wiki/Confidence_interval) para $\mu$. Cuando $\sigma$ [se desconoce](https://en.wikipedia.org/wiki/Normal_distribution#Confidence_intervals) necesitamos la $t$ de Student.

In [ ]:
# comprobaciones iniciales

from scipy.stats import t
from scipy.stats import norm

ts = t(5-1)

g = norm()

deltat = ts.ppf(0.5+.95/2)
deltat = ts.ppf(1-0.05/2)
print(deltat)

deltan = g.ppf(0.5+.95/2)
print(deltan)

In [ ]:
def exper(n, method='true'):
    mu1,mu2 =   -3,3
    sig1,sig2 = 0.2, 1
    mu =  np.random.rand()*(mu2-mu1) + mu1
    sig = np.random.rand()*(sig2-sig1) + sig1
    sample1 = np.random.randn(n)*sig + mu

    emu  = sample1.mean()
    esig = sample1.std()*n/(n-1)

    deltan = g.ppf(0.5+.95/2)
    deltat = t(n-1).ppf(0.5+.95/2)
    
    # comprobamos que la mu y sigma estimadas no engloban la proporción
    # hay que sacarlo a una comprobación desde fuera
    if method == 'check':
        sample2 = np.random.randn(10000)*sig + mu
        prop  = ((sample2 < mu + deltan*sig) & (sample2 > mu - deltan*sig)).mean()
        eprop = ((sample2 < emu + deltan*esig) & (sample2 > emu - deltan*esig)).mean()
        return prop, eprop
    
    if method=='true':
        # IC con el verdadero sigma
        ic1 = emu - deltan*sig/np.sqrt(n)
        ic2 = emu + deltan*sig/np.sqrt(n)        
        return ic1 < mu < ic2
    
    if method=='gaussian':
        # IC mal calculado con el sigma observado
        ic1 = emu - deltan*esig/np.sqrt(n)
        ic2 = emu + deltan*esig/np.sqrt(n)
        return ic1 < mu < ic2
    
    if method=='student':
        # IC bien calculado con la t de student
        ic1 = emu - deltat*esig/np.sqrt(n)
        ic2 = emu + deltat*esig/np.sqrt(n)
        return ic1 < mu < ic2
    
    raise NameError

El 95% de las muestras están en $\mu \pm 1.96 \sigma$, pero no entre $\bar \mu \pm 1.96 s$.

In [ ]:
exper(5, 'check')

El intervalo de confianza para $\mu$ se debe construir con la verdadera $\sigma$ y una gaussiana o con su estimación $s$ y la t de Student.

In [ ]:
np.mean([ exper(5,'true') for _ in range(10000) ])

In [ ]:
np.mean([ exper(5, 'gaussian') for _ in range(10000) ])

In [ ]:
np.mean([ exper(5, 'student') for _ in range(10000) ])

La alternativa es aplicar Bayes directamente marginalizando $\sigma$.

In [ ]:
n = 100
M = np.linspace(-3,3,n)
S = np.linspace(0.1,3,n+1)
#S = np.linspace(0.49,0.51,n+1)
s,m = np.meshgrid(S,M,indexing='ij')

def BayesInterval(D, show=False):
    L = sum([lgauss1d(m,s,d) for d in D]) # + ljeffreys(s)
    Pms = normalize(np.exp(L))
    Pm = Pms.sum(axis=0)
    Ps = Pms.sum(axis=1)
    m1,m2 = M[np.where(Pm > levels(Pm,[0.95])[0][0])[0][[0,-1]]]
    if show:
        plt.figure(figsize=(8,4))
        plt.subplot(1,2,1); plt.plot(M,Pm);
        shrange(m1,m2,d=Pm.max()/50)
        plt.subplot(1,2,2); plt.plot(S,Ps);
    return m1,m2

N = 5
mu = 1
sig = 0.5
dat = mu + sig*G(N)
BayesInterval(dat,show=True);

Cuando repetimos el experimento la media queda dentro el porcentaje de veces deseado. El procedimiento Bayesiano directo consigue "automáticamente" el efecto de usar la t de Student.

In [ ]:
N = 5
tot = 0
trials = 1000
for k in range(trials):
    dat = mu + sig*G(N)
    x1,x2 = BayesInterval(dat,show=False)
    tot += x1 < mu < x2
print(f'{tot/trials*100:.0f}%')

Pendiente: usar el generador con el prior utilizado.

### Predictive distributions

Y otra cuestión interesante es la de conseguir la distribución de nuevas muestras. Podemos construir una gaussiana con los estimadores más verosímiles $\bar x$ y $s$, o podemos marginalizar su distribución conjunta dados los datos. Esto va a ensanchar la distribución final, teniendo en cuenta automáticamente el tamaño de la muestra.

In [ ]:
def nice():
    ax = plt.gca()
    ax.spines['left'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_yticklabels([])
    ax.set_yticks([])
    ax.spines['bottom'].set_position('zero')
    ax.spines['bottom'].set_color('gray')
    
    col = 'gray'
    ax.tick_params(axis='x', colors=col)

In [ ]:
# trabaja con M,S,X,m,s globales
# FIXME: fijo a 95%
def BayesPredictInterval(D, method='good', show=False, truef=None):
    
    n = len(D)
    
    if n>1:
        emu  = D.mean()
        esig = D.std()*n/(n-1)
    
    if method=='bad':
        return emu-1.96*esig , emu+1.96*esig

    L = sum([lgauss1d(m,s,d) for d in D])

    x = X.reshape(-1, *np.ones(L.ndim,int))

    L = lgauss1d(m,s,x) + L
    
    P = normalize(np.exp(L))
    Px = P.sum(axis=(1,2))
    m1,m2 = X[np.where(Px > levels(Px,[0.95])[0][0])[0][[0,-1]]]
    
    if show:
        Pm = P.sum(axis=(0,1))
        Ps = P.sum(axis=(0,2))
        plt.figure(figsize=(12,4))
        plt.subplot(1,3,1); plt.plot(M,Pm); plt.xlabel('$\mu$'); plt.yticks([])
        plt.subplot(1,3,2); plt.plot(S,Ps); plt.xlabel('$\sigma$'); plt.yticks([])
        plt.subplot(1,3,3); plt.plot(X,Px,lw=3,label='Bayes')
        if truef is not None:
            plt.plot(X, normalize(truef.pdf(X)),label='true')
            plt.xlabel('$x$'); plt.yticks([])
        options = { 'marker': 'x', 's': 60, 'alpha': 0.50, 'color':'blue' }
        plt.scatter(D,n*[0], zorder=5, **options);
        _,_,y1,y2 = plt.axis()
        nice()
        if n>1:
            bad = norm(emu,esig)
            plt.plot(X, normalize(bad.pdf(X)),color='red',ls='dotted',label='ML')
        shrange(m1,m2,d=Px.max()/50)
        plt.legend()
        plt.ylim(y1,y2)
        plt.suptitle(f'N={n}')

    if method=='good':
        return m1, m2

    raise NameError

In [ ]:
def genprob(mus,sigmas,n = 100):
    mu1,mu2 = mus
    sig1,sig2 = sigmas
    x1 = mu1-3*sig2
    x2 = mu2+3*sig2
    
    M = np.linspace(mu1,mu2,n)
    S = np.linspace(sig1,sig2,n+1)
    X = np.linspace(x1,x2,n+5)
    
    def gen():
        mu =  np.random.rand()*(mu2-mu1) + mu1
        sig = np.random.rand()*(sig2-sig1) + sig1
        return mu,sig
    return M,S,X,gen

In [ ]:
M,S,X,gen = genprob((-1,1),(0.1,1))
s,m = np.meshgrid(S,M,indexing='ij')

Vemos que la distribución predictiva efectivamente engloba una muestra futura la proporción de veces deseada, mientras que la gaussiana estimada con los parámetros más verosímiles lo hace con bastante menos frecuencia. La primera es más ancha, pero solo lo justo, sin pasarse.

In [ ]:
mu,sig = gen()
BayesPredictInterval(mu + sig*G(3),show=True, truef=norm(mu,sig));

In [ ]:
N = 5
tot = 0
trials = 500
for k in range(trials):
    mu,sig = gen()
    dat = mu + sig*G(N)
    x1,x2 = BayesPredictInterval(dat)
    new = mu + sig*G(1)[0]
    tot += x1 < new < x2
print(f'{tot/trials*100:.0f}%')

In [ ]:
N = 5
tot = 0
trials = 500
for k in range(trials):
    mu,sig = gen()
    dat = mu + sig*G(N)
    x1,x2 = BayesPredictInterval(dat,method='bad')
    new = mu + sig*G(1)[0]
    tot += x1 < new < x2
print(f'{tot/trials*100:.0f}%')

In [ ]:
N = 2
tot = 0
trials = 500
for k in range(trials):
    mu,sig = gen()
    dat = mu + sig*G(N)
    x1,x2 = BayesPredictInterval(dat)
    new = mu + sig*G(1)[0]
    tot += x1 < new < x2
print(f'{tot/trials*100:.0f}%')

In [ ]:
N = 2
tot = 0
trials = 500
for k in range(trials):
    mu,sig = gen()
    dat = mu + sig*G(N)
    x1,x2 = BayesPredictInterval(dat,method='bad')
    new = mu + sig*G(1)[0]
    tot += x1 < new < x2
print(f'{tot/trials*100:.0f}%')

Y para nota: una sola observación:

In [ ]:
N = 1
tot = 0
trials = 500
for k in range(trials):
    mu,sig = gen()
    dat = mu + sig*G(N)
    x1,x2 = BayesPredictInterval(dat)
    new = mu + sig*G(1)[0]
    tot += x1 < new < x2
print(f'{tot/trials*100:.0f}%')

In [ ]:
mu,sig = gen()
BayesPredictInterval(mu + sig*G(1),show=True, truef=norm(mu,sig));

Podríamos hacer lo siguiente: suponer una media de cero y una sigma de 0.5, que son los valores medios de la información a priori para formar el intervalo. Pero no funciona:

In [ ]:
N = 1
tot = 0
trials = 500
for k in range(trials):
    mu,sig = gen()
    dat = mu + sig*G(N)
    x1,x2 = 0 - 1.96*0.5, 0 + 1.96*0.5
    new = mu + sig*G(1)[0]
    tot += x1 < new < x2
print(f'{tot/trials*100:.0f}%')

Tampoco usando como centro la muestra observada:

In [ ]:
N = 1
tot = 0
trials = 500
for k in range(trials):
    mu,sig = gen()
    dat = mu + sig*G(N)
    med = np.mean(dat)
    x1,x2 = med - 1.96*0.5, med + 1.96*0.5
    new = mu + sig*G(1)[0]
    tot += x1 < new < x2
print(f'{tot/trials*100:.0f}%')

Creo que no tiene mucho sentido evaluar esto con más de una muestra futura. Lo que se consiga será una "amplificación" de la calidad del intervalo, que es lo que queremos medir. Y estoy pensando que esto se puede hacer analíticamente. En lugar de sacar una nueva, ver la cantidad de la real que hay en el intervalo. Promediando esto en varias realizaciones posiblemente dé el mismo resultado.

Cuando hay sucesivas muestras futuras se podrían ir incorporando al estimador.

### Jeffreys

El problema precioso que sale en su libro: probabilidad de que $x_3$ caiga entre $x_1$ y $x_2$ (es 1/3), y de que $\mu$ caiga entre $x_1$ y $x_2$ (es 1/2).

In [ ]:
import numpy as np

x1,x2,x3 = np.random.randn(3,10000)
mn = np.min(np.array([x1,x2]),axis=0)
mx = np.max(np.array([x1,x2]),axis=0)

print( ((mn<0) & (0<mx)).mean() )

print( ((mn<x3) & (x3<mx)).mean() ) 

In [ ]:
import numpy as np

x1,x2,x3 = np.random.rand(3,10000)
mn = np.min(np.array([x1,x2]),axis=0)
mx = np.max(np.array([x1,x2]),axis=0)

print( ((mn<0.5) & (0.5<mx)).mean() )

print( ((mn<x3) & (x3<mx)).mean() )

print( (x3 > mx).mean() )

print( (x3 < mn).mean() )

No depende de la densidad concreta (eso sí, la localización debe ser la mediana). Vale para cualquier *location-scale parametric family*.

Esto le complica mucho la vida a los frecuentistas ya que no pueden definir distribuciones predictivas.

El primer caso, que el "true value" esté entre las dos muestras creo que es inmediato cuando el "true value" es la mediana, ya que por definición tienes 1/2 de probabilidad a cada lado. Hay 4 casos equiprobables para las dos muestras de caer a cada lado de la mediana, de las cuales 2 la engloban.

## Regresión

Estamos interesados en ganar intuición en la selección Bayesiana de modelos. Las técnicas de grid son una buena herramienta para ello, especialmente en el problema de regresión con modelos polinomiales de diferentes grados.

https://en.wikipedia.org/wiki/Bayes_factor

Una conclusión importante obtenida *the hard way* es que para que el contraste de modelos sea correcto hay que tener en cuenta explícitamente las probabilidades *a priori* de los parámetros. Si añades parámetros pero cambias el rango de alguno de los anteriores el espacio explorado cambia en dos sentidos. Por un lado aumenta la dimensión, pero a la vez puede aumentar o disminuir el tamaño de las capas.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from numpy import log

def normalize(x):
    return x/x.sum()

def lgauss1d(m, s, x):
    return -0.5 * ((x-m)/s)**2 - log(s)

%pip install -q https://raw.githubusercontent.com/albertoruiz/jupyterlite/main/content/misc/umucv-0.3-py3-none-any.whl

import umucv.prob as pr

### Datos

In [ ]:
plt.figure(figsize=(6,6))
X = np.arange(8)
Y = 2 + 1/3*X + np.random.randn(len(X))/2

plt.plot(X,Y,'.'); plt.axis('equal');

### Modelo lineal básico

Creamos el bloque exhaustivo de parámetros con `meshgrid`.

<p style='margin-left:2cm; color:#444; line-height:1.2'><small><small>**Nota**: El bloque se puede construir con bucles o *list comprehensions* pero numpy es mucho más rápido. Sin embargo, para ahorrar memoria, no merece la pena generar bloques replicados para los datos, que se suman inmediatamente.
</small></small></p>

In [ ]:
a1,a2 = -1  , 2
b1,b2 = -1  , 5
s1,s2 =  0.1, 2

In [ ]:
n = 75
A = np.linspace(a1,a2,n)
B = np.linspace(b1,b2,n)
S = np.linspace(s1,s2,n)

dv = (a2-a1) * (b2-b1) * (s2-s1) / n**3

a,b,s = np.meshgrid(A,B,S, indexing='ij')

L = 0
for x,y in zip(X,Y):
    L += lgauss1d( a*x + b , s , y )

L += -log(a2-a1) -log(b2-b1) -log(s2-s1)
    
P = normalize(np.exp(L))

evi_lin = np.exp(L).sum() * dv

Algunas distribuciones marginales:

In [ ]:
plt.figure(figsize=(12,4))
plt.subplot(1,3,1)
plt.plot(A,P.sum(axis=(1,2)));
plt.subplot(1,3,2)
plt.plot(B,P.sum(axis=(0,2)));
plt.subplot(1,3,3)
plt.plot(S,P.sum(axis=(0,1)));

In [ ]:
plt.imshow(P.sum(axis=2),'gray');
ax = plt.gca()
ax.set_xticks([0,len(B)-1]); ax.set_xticklabels(B[[0,-1]]);
ax.set_yticks([0,len(A)-1]); ax.set_yticklabels(A[[0,-1]]);

El modelo más verosímil, marginalizando el ruido:

In [ ]:
am,bm,_ = [ x[k] for x,k in zip([A,B,S], np.unravel_index(np.argmax(L),L.shape)) ]

x1 = min(X)-2
x2 = max(X)+2

plt.plot(X,Y,'.'); plt.axis('equal');
plt.plot([x1,x2],[am*x1+bm,am*x2+bm]);

Y un muestreo de los modelos:

In [ ]:
def sample():
    Pab = P.sum(axis=2)

    v,p = zip(*np.ndenumerate(Pab))
    lines = [(A[v[k][0]], B[v[k][1]]) for k in  np.random.choice(len(v),p=p,size=100)]

    for a,b in lines:
        plt.plot([x1,x2],[a*x1+b,a*x2+b],color='black', lw=3, alpha=0.05);
    #plt.plot([x1,x2],[am*x1+bm,am*x2+bm],'yellow');
    plt.plot(X,Y,'.',color='red'); plt.axis('equal');

sample()

Finalmente, la predicción en un $x$ concreto.

In [ ]:
YP = np.linspace(1,8,20)
#u = np.ones(L.shape)
#yp = np.outer(YP,u).reshape(*[-1]+list(u.shape))
yp = YP.reshape(-1, *np.ones(L.ndim,int))
xp = 8

PP = normalize(np.exp((lgauss1d(a*xp +b , s , yp ) + L )))
print(PP.shape)

dlin =  PP.sum(axis=(1,2,3))

plt.plot(YP,dlin);
pr.showhdi(pr.P(dict(zip(YP,dlin))), 90)
print(am*xp+bm)

<p style='text-align:right; margin-left:10cm; color:#444; line-height:1.2'><small><small>Creo que aquí no se puede hacer ningún atajo, necesitamos la expansión completa de todos los $y$ para poder normalizar y marginalizar correctamente. (Aunque tengo que comprobarlo, no estoy seguro.)
</small></small></p>

### Modelo cuadrático

In [ ]:
n = 50

c1, c2 = -1/5 , 1/5

A = np.linspace(a1,a2,n)
B = np.linspace(b1,b2,n)
S = np.linspace(s1,s2,n)
C = np.linspace(c1,c2,n)

dv = (a2-a1) * (b2-b1) * (s2-s1) * (c2-c1) / n**4

a,b,c,s = np.meshgrid(A,B,C,S, indexing='ij')

L = 0
for x,y in zip(X,Y):
    L += lgauss1d( c*x**2 + a*x + b , s , y )
    
# L*= 0  # to check prior
L +=  -log(a2-a1) -log(b2-b1) -log(s2-s1) -log(c2-c1)    

P = normalize(np.exp(L))

evi_cua = np.exp(L).sum() * dv

In [ ]:
evi_lin, evi_cua, evi_lin/evi_cua

In [ ]:
plt.figure(figsize=(16,4))
plt.subplot(1,4,1)
plt.plot(A,P.sum(axis=(1,2,3)));
plt.subplot(1,4,2)
plt.plot(B,P.sum(axis=(0,2,3)));
plt.subplot(1,4,3)
plt.plot(C,P.sum(axis=(0,1,3)));
plt.subplot(1,4,4)
plt.plot(S,P.sum(axis=(0,1,2)));

In [ ]:
plt.imshow(P.sum(axis=(2,3)),'gray');
ax = plt.gca()
ax.set_xticks([0,len(B)-1]); ax.set_xticklabels(B[[0,-1]]);
ax.set_yticks([0,len(A)-1]); ax.set_yticklabels(A[[0,-1]]);

In [ ]:
am,bm,cm,_ = [ x[k] for x,k in zip([A,B,C,S], np.unravel_index(np.argmax(L),L.shape)) ]

plt.plot(X,Y,'.'); plt.axis('equal');
xplot = np.linspace(x1,x2,50)
yplot = [cm*x**2 + am*x + bm for x in xplot]
plt.plot(xplot,yplot);

Y un muestreo de los modelos:

In [ ]:
def sample():
    Pabc = P.sum(axis=3)

    v,p = zip(*np.ndenumerate(Pabc))
    lines = [(A[v[k][0]], B[v[k][1]], C[v[k][2]]) for k in  np.random.choice(len(v),p=p,size=100)]

    plt.figure(figsize=(4,6))
    for a,b,c in lines:
        yplot = [c*x**2 + a*x + b for x in xplot]
        plt.plot(xplot,yplot,color='black', lw=3, alpha=0.05);
    plt.plot(X,Y,'.',color='red'); plt.axis('equal');

sample()

In [ ]:
YP = np.linspace(1,8,20)
#u = np.ones(L.shape)
#yp = np.outer(YP,u).reshape(*[-1]+list(u.shape))
yp = YP.reshape(-1, *np.ones(L.ndim,int))
xp = 8

PP = normalize(np.exp((lgauss1d(c*xp**2 + a*xp + b , s , yp ) + L )))
print(PP.shape)

dcua =  PP.sum(axis=(1,2,3,4))

plt.plot(YP, dcua);
pr.showhdi(pr.P(dict(zip(YP,dcua))), 90)
print(bm + am*xp + cm*xp**2)

In [ ]:
plt.plot(YP,dlin, YP, dcua);

### Modelo más simple

In [ ]:
n = 120

A = np.linspace(a1,a2,n)
S = np.linspace(s1,s2,n)

dv = (a2-a1) * (s2-s1) / n**2

a,s = np.meshgrid(A,S, indexing='ij')

bf = 1

L = 0
for x,y in zip(X,Y):
    L += lgauss1d( a*x + bf , s , y )

L +=  -log(a2-a1) -log(s2-s1)
    
P = normalize(np.exp(L))

evi_simp = np.exp(L).sum() * dv

In [ ]:
evi_lin, evi_simp, evi_lin/evi_simp

In [ ]:
plt.figure(figsize=(16,4))
plt.subplot(1,4,1)
plt.plot(A,P.sum(axis=(1)));
plt.subplot(1,4,4)
plt.plot(S,P.sum(axis=(0)));

In [ ]:
am,_ = [ x[k] for x,k in zip([A,S], np.unravel_index(np.argmax(L),L.shape)) ]

plt.plot(X,Y,'.'); plt.axis('equal');
xplot = np.linspace(x1,x2,50)
yplot = [am*x + bf for x in xplot]
plt.plot(xplot,yplot); plt.grid();

Y un muestreo de los modelos:

In [ ]:
def sample():
    Pa = P.sum(axis=1)

    v,p = zip(*np.ndenumerate(Pa))
    lines = [(A[v[k][0]]) for k in  np.random.choice(len(v),p=p,size=100)]

    for a in lines:
        yplot = [a*x + bf for x in xplot]
        plt.plot(xplot,yplot,color='black', lw=3, alpha=0.05);
    plt.plot(X,Y,'.',color='red'); plt.axis('equal');
    
sample()    

## Función Q

Eliminación analítica del parámetro $\sigma$.

Muchos modelos consisten en una cierta ley que depende de parámetros $\theta$, y las observaciones tienen un ruido gaussiano $\sigma$ en principio desconocido. Si tenemos $N$ datos $D$, lo único importante es $Q(\theta)$, la suma de errores cuadráticos. 

$$p(D|\theta \sigma) \propto  \frac{1}{\sigma^N} \exp\left(-\frac{Q(\theta)}{2\sigma^2}\right)$$

Con prior no informativa para $\sigma$, Bayes nos dice:

$$p(\theta \sigma|D) \propto p(\sigma) p(\theta) p(D|\theta \sigma)  \propto \frac{1}{\sigma}\, p(\theta)\, \frac{1}{\sigma^N} \exp\left(-\frac{Q(\theta)}{2\sigma^2}\right) $$

Marginalizando $\sigma$ y evaluando la integral simbólica (p. ej. con sympy), llegamos a:

$$p(\theta |D) \propto p(\theta) \int_0^\infty d\sigma  \frac{1}{\sigma^{N+1}} \exp\left(-\frac{Q(\theta)}{2\sigma^2}\right) \propto p(\theta) Q(\theta)^{\frac{-N}{2}}$$



Para la comprobación hay que usar, por supuesto el prior no informativo para $\sigma$. Si usamos el uniforme, como en ejemplos anteriores, se produce una leve discrepancia en las marginales. Lo que confirma la consistencia de las expresiones.

La estimación de la distribución a posteriori de sigma no es exacta pero sí una buena aproximación, que mejora con $N$:

$$
p(\sigma | D) \sim \frac{1}{\sigma^{N+1}} \exp\left(-\frac{Q_{min}}{2\sigma^2} \right)$$

In [ ]:
n = 50

c1, c2 = -1/5 , 1/5

A = np.linspace(a1,a2,n)
B = np.linspace(b1,b2,n)
S = np.linspace(s1,s2,n)
C = np.linspace(c1,c2,n)

dv = (a2-a1) * (b2-b1) * (s2-s1) * (c2-c1) / n**4

a,b,c,s = np.meshgrid(A,B,C,S, indexing='ij')

L = 0
for x,y in zip(X,Y):
    L += lgauss1d( c*x**2 + a*x + b , s , y )

L +=  -log(a2-a1) -log(b2-b1) -log(s) -log(c2-c1)    

P = normalize(np.exp(L))

evi_cua = np.exp(L).sum() * dv

In [ ]:
a,b,c = np.meshgrid(A,B,C, indexing='ij')

Q = 0
for x,y in zip(X,Y):
    Q += ( y - (c*x**2 + a*x + b) ) ** 2

N = len(X)
    
lnp = -N/2 *log(Q)
lnp += -log(a2-a1) -log(b2-b1) -log(c2-c1)    
lnp -= lnp.max()
P2  = normalize(np.exp(lnp))

In [ ]:
plt.figure(figsize=(16,4))
plt.subplot(1,4,1)
plt.plot(A,P.sum(axis=(1,2,3)),lw=5);
plt.plot(A,P2.sum(axis=(1,2)));
plt.subplot(1,4,2)
plt.plot(B,P.sum(axis=(0,2,3)),lw=5);
plt.plot(B,P2.sum(axis=(0,2)));
plt.subplot(1,4,3)
plt.plot(C,P.sum(axis=(0,1,3)),lw=5);
plt.plot(C,P2.sum(axis=(0,1)));
plt.subplot(1,4,4)
plt.plot(S,normalize(P.sum(axis=(0,1,2))));
pS = normalize(1/S**(len(X)+1)*np.exp(-Q.min()/2/S**2))
plt.plot(S, pS);

In [ ]:
plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
plt.imshow(1-P.sum(axis=(2,3)),'gray');
ax = plt.gca()
ax.set_xticks([0,len(B)-1]); ax.set_xticklabels(B[[0,-1]]);
ax.set_yticks([0,len(A)-1]); ax.set_yticklabels(A[[0,-1]]);
plt.subplot(1,2,2)
plt.imshow(1-P2.sum(axis=(2)),'gray');
ax = plt.gca()
ax.set_xticks([0,len(B)-1]); ax.set_xticklabels(B[[0,-1]]);
ax.set_yticks([0,len(A)-1]); ax.set_yticklabels(A[[0,-1]]);

La coincidencia es perfecta en la marginalización de $\sigma$, y la aproximación a $p(\sigma|D)$ es muy buena.

Queda pendiente la selección de modelos usando esta técnica. Una forma sería elegir el $\sigma$ más probable de cada modelo y eliminarlo del modelo. Por cierto, ¿el exponente es N+1? en el caso anterior ajusta mejor N-1.